In [ ]:
import sys

!test -d or-gym || git clone https://github.com/hubbs5/or-gym.git or-gym
if not 'or-gym' in sys.path:
    sys.path += ['or-gym']

!cd or-gym && pip install -q --editable .

Cloning into 'or-gym'...
remote: Enumerating objects: 2749, done.
remote: Counting objects: 100% (212/212), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 2749 (delta 114), reused 150 (delta 75), pack-reused 2537
Receiving objects: 100% (2749/2749), 24.91 MiB | 30.85 MiB/s, done.
Resolving deltas: 100% (1659/1659), done.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [ ]:
!pip install ray
#!pip install numpy==1.19.5

     |████████████████████████████████| 49.6 MB 14 kB/s 
     |████████████████████████████████| 72 kB 485 kB/s 


In [ ]:
# pip install 'ray[rllib]'

In [ ]:
pip install ray[Tune]

     |████████████████████████████████| 124 kB 5.2 MB/s 


In [ ]:
import or_gym
from or_gym.utils import create_env
import ray
from ray.rllib import agents
from ray import tune
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.


In [ ]:
'''
# set default (arbitrary) values when creating environment (if no args or kwargs are given)
self.periods = 30   # [positive integer] number of periods in simulation.
self.I0 = [100, 100, 200]   # [non-negative integer; dimension |Stages|-1] initial inventories for each stage.
self.p = 2    # [positive float] unit price for final product.
self.r = [1.5, 1.0, 0.75, 0.5]    #  [non-negative float; dimension |Stages|] unit cost for replenishment orders at each stage.
self.k = [0.10, 0.075, 0.05, 0.025]   # [non-negative float; dimension |Stages|] backlog cost or goodwill loss (per unit) for unfulfilled orders (demand or replenishment orders).
self.h = [0.15, 0.10, 0.05] # [non-negative float; dimension |Stages|-1] unit holding cost for excess on-hand inventory at each stage.(Note: does not include pipeline inventory).
self.c = [100, 90, 80]    # [positive integer; dimension |Stages|-1] production capacities for each suppliers (stages 1 through |Stage|).
self.L = [3, 5, 10]     # [non-negative integer; dimension |Stages|-1] lead times in betwen stages.
self.backlog = True
self.dist = 1
self.dist_param = {'mu': 20}
self.alpha = 0.97
self.seed_int = 0
self.user_D = np.zeros(self.periods)
self._max_rewards = 2000
'''

"\n# set default (arbitrary) values when creating environment (if no args or kwargs are given)\nself.periods = 30   # [positive integer] number of periods in simulation.\nself.I0 = [100, 100, 200]   # [non-negative integer; dimension |Stages|-1] initial inventories for each stage.\nself.p = 2    # [positive float] unit price for final product.\nself.r = [1.5, 1.0, 0.75, 0.5]    #  [non-negative float; dimension |Stages|] unit cost for replenishment orders at each stage.\nself.k = [0.10, 0.075, 0.05, 0.025]   # [non-negative float; dimension |Stages|] backlog cost or goodwill loss (per unit) for unfulfilled orders (demand or replenishment orders).\nself.h = [0.15, 0.10, 0.05] # [non-negative float; dimension |Stages|-1] unit holding cost for excess on-hand inventory at each stage.(Note: does not include pipeline inventory).\nself.c = [100, 90, 80]    # [positive integer; dimension |Stages|-1] production capacities for each suppliers (stages 1 through |Stage|).\nself.L = [3, 5, 10]     #

In [ ]:
# env_config = {'dist':6,
#               'dist_param' : {'scale':2,'loc':20}}
env_config = {}
# env_config = {'periods': 50,
#           'I0':[100,100,100,200],
#           'r':[2.50, 2.00,1.00,0.75,0.50],
#           'k':[0.15,0.10,0.075,0.05, 0.025],
#           'h':[0.3,0.15,0.10,0.05],
#           'c':[130,100,90,80],
#           'L':[5,7,10,15],
#           'max_rewards':1000}
# env_config = {'periods': 10,
#           'I0':[100,200],
#           'r':[1.00,0.75,0.50],
#           'k':[0.075,0.05, 0.025],
#           'h':[0.10,0.05],
#           'c':[90,80],
#           'L':[3,5],
#           'max_rewards':1000}
env_name = 'InvManagement-v1'
f=open("default ppo default.txt","w")
# env = or_gym.make('InvManagement-v1', env_config=env_config)



In [ ]:
def register_env(env_name, env_config={}):
    env = create_env(env_name)
    tune.register_env(env_name, 
        lambda env_name: env(env_name,
            env_config=env_config))

In [ ]:
rl_config = dict(
    env=env_name,
    num_workers=2,
    env_config=env_config,
    train_batch_size = 1000,

    # model=dict(
    #     # vf_share_layers=False,
    #     fcnet_activation='elu',
    #     fcnet_hiddens=[256, 256],
    #     lstm_cell_size = 64,
    #     # max_seq_len =  20,
    #     vf_share_layers= False,
    #     use_lstm=True,
    #     lstm_use_prev_action = True,
    #     lstm_use_prev_reward = True,
    # ),
    # lr=1e-2
    
    #default ppo parameters
    model=dict(
        vf_share_layers=False,
        fcnet_activation='elu',
        fcnet_hiddens=[256, 256]
    ),
    lr=1e-5
)
# Register environment
register_env(env_name, env_config)



In [ ]:
ray.init(ignore_reinit_error=True)
agent = agents.ppo.PPOTrainer(env=env_name, config=rl_config)
results = []
for i in range(550):
    res = agent.train()
    results.append(res)
    f.write(str(res['episode_reward_mean']))
    f.write("\n")
    print('\rIter: {}\tReward: {:.2f}'.format(i + 1, res['episode_reward_mean']), end='')

/usr/local/lib/python3.7/dist-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-08-24 23:57:59,409	INFO trainer.py:714 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-08-24 23:57:59,411	WARNING ppo.py:143 -- `train_batch_size` (1000) cannot be achieved with your other settings (num_workers=2 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 500.
2021-08-24 23:57:59,413	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-08-24 23:57:59,421	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or

Iter: 550	Reward: 453.37

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
 
# Unpack values from each iteration
rewards = np.hstack([i['hist_stats']['episode_reward'] 
    for i in results])
# pol_loss = [
#     i['info']['learner']['default_policy']['policy_loss'] for i in results]
# vf_loss = [
#     i['info']['learner']['default_policy']['vf_loss'] 
#     for i in results]
 
p = 100
mean_rewards = np.array([np.mean(rewards[i-p:i+1]) 
                if i >= p else np.mean(rewards[:i+1]) 
                for i, _ in enumerate(rewards)])
std_rewards = np.array([np.std(rewards[i-p:i+1])
               if i >= p else np.std(rewards[:i+1])
               for i, _ in enumerate(rewards)])

fig = plt.figure(constrained_layout=True, figsize=(20, 10))
gs = fig.add_gridspec(2, 4)
ax0 = fig.add_subplot(gs[:, :-2])
ax0.fill_between(np.arange(len(mean_rewards)), 
                 mean_rewards - std_rewards, 
                 mean_rewards + std_rewards, 
                 label='Standard Deviation', alpha=0.3)
ax0.plot(mean_rewards, label='Mean Rewards')
ax0.set_ylabel('Rewards',fontsize=20)
ax0.set_xlabel('Episode',fontsize=20)
ax0.set_title('LSTM+PPO (default state vector + 5 stages) (discrete)', fontsize = 17)
ax0.text
ax0.legend(loc = 4)
plt.yticks(fontsize = 15)
plt.xticks(fontsize = 15)
# plt.ylim([-300, 500])
plt.grid()
 
# ax1 = fig.add_subplot(gs[0, 2:])
# ax1.plot(pol_loss)
# ax1.set_ylabel('Loss')
# ax1.set_xlabel('Iteration')
# ax1.set_title('Policy Loss')
 
# ax2 = fig.add_subplot(gs[1, 2:])
# ax2.plot(vf_loss)
# ax2.set_ylabel('Loss')
# ax2.set_xlabel('Iteration')
# ax2.set_title('Value Function Loss')

plt.show()

<Figure size 1440x720 with 1 Axes>


In [ ]:
x = np.hstack([i['episode_reward_mean'] 
    for i in results])
print(x)
f.write("1231234124124124")

[-50.31228548  -7.98596737  27.77829059  75.9633751  105.37748209
 125.06710911 145.15631676 163.64269    178.64587193 195.2963669
 209.86072028 210.95576095 209.53517701 208.35785857 223.7774969
 240.13432058 252.39902309 256.51754938 256.07401555 262.12241826
 261.09529551 264.39478709 270.01030156 276.73423531 288.83594986
 302.18148053 313.79305548 321.36350317 321.80225837 325.4316903
 328.28744786 334.61125669 342.04380714 340.07677887 339.28012031
 339.50609005 344.50390518 348.20686916 346.30024446 348.94006975
 349.83321143 350.34284941 347.93375271 344.43013024 345.09496436
 349.76071669 346.4626225  355.80469155 360.55680892 371.14157346
 375.50368778 377.66264995 381.22042658 377.41271968 380.65317705
 384.70052355 389.23959044 379.51296819 378.1470011  378.07726971
 383.3456547  386.14916269 391.32124975 389.78735005 387.98658857
 377.38831962 381.0943569  383.78764204 395.90213639 403.26600124
 403.90266227 401.30258612 397.7843437  391.59535735 391.13426323
 388.74425948

16

In [ ]:
with open('Example2.txt', 'w') as writefile:
  for i in x:

    writefile.write(str(i))
    writefile.write("\n")